In [199]:
import os
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn.functional as F
from PIL import Image, ImageFilter
from sklearn.model_selection import train_test_split
from torch import nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torchvision import transforms
from pathlib import Path

In [200]:
def unzip_dataset(INPATH, OUTPATH):
    with zipfile.ZipFile(INPATH) as zf:
        zf.extractall(OUTPATH)
    
unzip_dataset(INPATH = './train_data.zip', OUTPATH = './')
unzip_dataset(INPATH = './test_data.zip', OUTPATH = './')

In [201]:
train_dir = '../Casting_products_pytorch/train_data/'
test_dir = '../Casting_products_pytorch/test_data/'
train_df = pd.read_csv('../Casting_products_pytorch/train.csv')

In [202]:
train_df, val_df = train_test_split(np.array(train_df), test_size=0.1)

In [203]:
transform = torchvision.transforms.ToTensor()

In [204]:
class Casting_products_Datasets(Dataset):
    def __init__(self, dir_name, csv_file, transform=None):
        self.dir_name = dir_name
        self.csv_file = csv_file
        self.transform = transform

    def __len__(self):
        return len(self.csv_file)
    
    def __getitem__(self, index):
        file = self.csv_file[index][0]
        label = np.array([self.csv_file[index][1]])
        image = Image.open(self.dir_name + file)
        image = self.transform(image)

        return image, label

In [205]:
train_dataset = Casting_products_Datasets(train_dir, train_df, transform)
val_dataset = Casting_products_Datasets(train_dir, val_df, transform)

In [206]:
batch_size = 5

In [207]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [223]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=12)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(num_features=24)
        self.conv5 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(num_features=24)
        self.fc1 = nn.Linear(24*5*5, 128)
        self.fc2 = nn.Linear(128, 48)
        self.fc3 = nn.Linear(48, 2)
    
    def forward(self, x):
        
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.relu(self.bn5(self.conv5(x)))
        x = x.view(-1, 24*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x
    
model = Network()

In [224]:
loss_fnc = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())
record_loss_train = []
record_loss_val = []

In [225]:
images, labels = iter(val_dataloader).next()

In [226]:
len(labels)

5

In [227]:
for i in range(20):
    model.train()
    loss_train = 0
    for j, (x, t) in enumerate(train_dataloader):
        y = model(x)
        loss = loss_fnc(y, t)
        loss_train += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_train /= j+1
    record_loss_train.append(loss_train)

    model.eval()  # 評価モード
    y_test = model(images)
    loss_test = loss_fnc(y_test, images).item()
    record_loss_val.append(loss_test)

    if i%1 == 0:
        print("Epoch:", i, "Loss_Train:", loss_train, "Loss_Test:", loss_test)


RuntimeError: shape '[-1, 600]' is invalid for input of size 2488320